In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import curve_fit

from scipy.stats import kde
from scipy import optimize

from matplotlib.ticker import NullFormatter
from matplotlib import pyplot, transforms
import matplotlib
from matplotlib.ticker import AutoMinorLocator
%matplotlib inline
import os
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

pd.set_option("display.max_columns", 300)

In [2]:
df = pd.read_csv('/Users/hkromer/02_PhD/04_Publiations/08.Defense/Figures/COMSOL_ion_spot.data', index_col=0)

In [3]:
df_y = pd.read_csv('/Users/hkromer/02_PhD/04_Publiations/08.Defense/Figures/COMSOL_ion_spot_y.data', index_col=0)
df_z = pd.read_csv('/Users/hkromer/02_PhD/04_Publiations/08.Defense/Figures/COMSOL_ion_spot_z.data', index_col=0)

%matplotlib widget
fs=21
f = plt.figure(1, figsize=(8, 6.4))
ylims = (-0.01, 0.21)
# top plot
plt.subplot(2, 1, 1)
plt.plot(df_y['y'].values, df_y['eval_y'].values, color='#1269b0', linewidth=2.0, label='z=0.56 mm')
# plt.plot(df_y['y'].values, df_y['Y_fit_y'].values, '-.',color='#b03412',linewidth=2.0, label='Gaussian fit')
plt.xlabel(r"\textbf{y [mm]}", color='gray',fontsize=fs)
ax = plt.gca()
ax.tick_params('x', colors='k', labelsize=fs-2)
ax.tick_params('y', colors='k', labelsize=fs-2)
plt.ylabel(r"\textbf{Estimated PDF [mm]}", color='gray', rotation=0, fontsize=fs)
ax.yaxis.set_label_coords(-0.05,1.1)
plt.legend(fontsize=fs-6)
ylim = ax.get_ylim()
plt.ylim(ylim[0], 0.225)

plt.subplot(2, 1, 2)
plt.plot(df_z['z'].values, df_z['eval_z'].values, color='#b03412', linewidth=2.0, label='y=0 mm')
# plt.plot(df_z['z'].values, df_z['Y_fit_z'].values, '-.',color='#b03412',linewidth=2.0, label='Gaussian fit')
plt.xlabel(r"\textbf{z [mm]}", color='gray',fontsize=fs)
ax = plt.gca()
ax.tick_params('x', colors='k', labelsize=fs-2)
ax.tick_params('y', colors='k', labelsize=fs-2)
plt.tight_layout()
plt.legend(fontsize=fs-6)
ylim = ax.get_ylim()
plt.ylim(ylim[0], 0.225)
plt.savefig('/Users/hkromer/02_PhD/04_Publiations/08.Defense/Figures/COMSOL_ion_spot_xy.pdf')
plt.savefig('/Users/hkromer/02_PhD/04_Publiations/08.Defense/Figures/COMSOL_ion_spot_xy.png', dpi=1500)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# maximum point for y plot
max_idx = np.argmax(df_z['eval_z'].values)
df_z['z'].values[max_idx]

0.5757575757575757

In [5]:
import scipy

x = df["qy"].values
y = df["qz"].values
%matplotlib widget

# Make a 2d normed histogram
H,xedges,yedges=np.histogram2d(x,y,bins=40,normed=True)

norm=H.sum() # Find the norm of the sum
# Set contour levels
contour1=0.95
contour2=0.50
# contour3=0.99

# Set target levels as percentage of norm
target1 = norm*contour1
target2 = norm*contour2
# target3 = norm*contour3

# Take histogram bin membership as proportional to Likelihood
# This is true when data comes from a Markovian process
def objective(limit, target):
    w = np.where(H>limit)
    count = H[w]
    return count.sum() - target

# Find levels by summing histogram to objective
level1= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target1,))
level2= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target2,))
# level3= scipy.optimize.bisect(objective, H.min(), H.max(), args=(target3,))

# For nice contour shading with seaborn, define top level
level4=H.max()
# levels=[level1,level2,level3,level4]
levels=[level1,level2, level4]
print(levels)

[0.08010215956154754, 0.16020431912426072, 0.3204086382508527]


In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
nbins = 200
lim = 3
#     print(data[pd.isnull(data).any(axis=1)])
x = df['qy'].values
y = df['qz'].values
my_data = np.vstack([x, y])
k = kde.gaussian_kde(my_data)

xi, yi = np.mgrid[-lim:lim:nbins*1j, -lim:lim:nbins*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))
%matplotlib widget

fig, ax = plt.subplots(figsize=(8,8))
p = sns.kdeplot(df["qy"].values, df["qz"].values, cmap='magma', shade=True, cbar=True)
plt.axis('equal')
ax.set_facecolor('black')
# ax.set_xlim(-2,2)
# ax.set_ylim(-3,3)
contours = ax.contour(xi, yi, zi.reshape(xi.shape), [level2], colors='black')
ax.clabel(contours, inline=True, fontsize=12, fmt='0.50')
contours = ax.contour(xi, yi, zi.reshape(xi.shape), [level1], colors='black')
ax.clabel(contours, inline=True, fontsize=12, fmt='0.95')
ax.set_yticks(np.arange(-3,4,1))
ax.set_xticks(np.arange(-3,4,1))
plt.xlim(-2.5,2.5)
plt.ylim(-2.5,2.5)
ax.collections[0].set_alpha(0)

ax.figure.axes[-1].tick_params(labelsize=fs-2)
ax.set_xlabel(r"\textbf{y [mm]}", color='gray',fontsize=fs)
ax.set_ylabel(r"\textbf{z [mm]}", color='gray', rotation=0, fontsize=fs)
ax.yaxis.set_label_coords(0,1.03)
ax.tick_params('x', colors='k', labelsize=fs-2)
ax.tick_params('y', colors='k', labelsize=fs-2)

plt.savefig('/Users/hkromer/02_PhD/04_Publiations/08.Defense/Figures/COMSOL_ion_spot_density.pdf')
plt.savefig('/Users/hkromer/02_PhD/04_Publiations/08.Defense/Figures/COMSOL_ion_spot_density.png', dpi=1200)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(6,6))

sns.kdeplot(df["qy"].values, df["qz"].values, cmap='viridis', cbar=True)
plt.hist2d(df["qy"].values, df["qz"].values, bins=20, cmin=1e-100, cmap="jet", alpha=0.4)
# ax.collections[0].set_alpha(0)
ax.set_facecolor('lightblue')
plt.axis('equal')

ax = plt.gca()
ax.set_yticks(np.arange(-3,4,1))
ax.set_xticks(np.arange(-3,4,1))
plt.xlim(-2.5,2.5)
plt.ylim(-2.5,2.5)
ax.figure.axes[-1].tick_params(labelsize=fs-2)
ax.set_xlabel(r"\textbf{y [mm]}", color='gray',fontsize=fs)
ax.set_ylabel(r"\textbf{z [mm]}", color='gray', rotation=0, fontsize=fs)
ax.yaxis.set_label_coords(0,1.03)
ax.tick_params('x', colors='k', labelsize=fs-2)
ax.tick_params('y', colors='k', labelsize=fs-2)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(6,6))

# sns.kdeplot(df["qy"].values, df["qz"].values, cmap='viridis', cbar=True)
h = plt.hist2d(df["qy"].values, df["qz"].values, bins=20, cmin=1e-100, cmap="jet", alpha=1)

plt.colorbar(h[3], ax=ax)
# ax.collections[0].set_alpha(0)
# ax.set_facecolor('lightblue')
plt.axis('equal')

ax = plt.gca()
ax.set_yticks(np.arange(-3,4,1))
ax.set_xticks(np.arange(-3,4,1))
plt.xlim(-2.5,2.5)
plt.ylim(-2.5,2.5)
ax.figure.axes[-1].tick_params(labelsize=fs-2)
ax.set_xlabel(r"\textbf{y [mm]}", color='gray',fontsize=fs)
ax.set_ylabel(r"\textbf{z [mm]}", color='gray', rotation=0, fontsize=fs)
ax.yaxis.set_label_coords(0,1.03)
ax.tick_params('x', colors='k', labelsize=fs-2)
ax.tick_params('y', colors='k', labelsize=fs-2)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …